In [105]:
import os
import math
import numpy as np
import pandas as pd
import datetime
from datetime import date
import pandas_ta as ta
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime
import fix_yahoo_finance as yf

In [106]:
def calculate_smas(df):
    df['SMA10'] = ta.sma(df["CRYPTO_INDEX"], length=10)
    df['SMA30'] = ta.sma(df["CRYPTO_INDEX"], length=30)

    return df

In [107]:
def determine_primary_market(df):
    df['BEAR'] = np.where(df['SMA10'] < df["SMA30"], df['CRYPTO_INDEX'], np.NaN)
    df['BULL'] = np.where(df['SMA10'] > df["SMA30"], df['CRYPTO_INDEX'], np.NaN) 

    return df

In [108]:
#  Index Calculation
def load_crypto_data(symbols):
    today = datetime.date.today()
    today_str = today.strftime("%Y-%m-%d")
    #  Get last year's data
    start_date = today - (600 * US_BUSINESS_DAY)
    start_date_str = datetime.datetime.strftime(start_date, "%Y-%m-%d")
    # Download data from Yahoo Finance
    try:
        df = si.get_data(symbols, start_date=start_date_str, end_date=today_str, index_as_date=False)
        return df
    except:
        print('Error loading stock data for ' + symbols)
        return None

In [109]:
symbols = ['XLM-USD','DOGE-USD','STX-USD']

start = datetime.datetime(2020,5,1)
end = datetime.datetime(2022,4,29)
crypto_df = yf.download(symbols, start=start, end=end)

#  Convert columns from tuple to strings
crypto_df.columns = ['{}_{}'.format(x[0], x[1]) for x in crypto_df.columns]
column_names = ['Close_STX-USD', 'Close_DOGE-USD', 'Close_XLM-USD']  
crypto_df['CRYPTO_INDEX']= crypto_df[column_names].sum(axis=1) / len(symbols)
crypto_df = calculate_smas(crypto_df)
crypto_df = determine_primary_market(crypto_df)

[*********************100%***********************]  3 of 3 completed


In [110]:
def plot_crypto_index(df):
    fig = make_subplots(rows=1, cols=1)

    #  Plot close price
    #fig.add_trace(go.Line(x = df.index, y = df['CRYPTO_INDEX'], line=dict(color="blue", width=1), name="Crypto Index"), row = 1, col = 1)
    
    #  Plot SMAs
    fig.add_trace(go.Line(x = df.index, y = df['SMA30'], line=dict(color="blue", width=1), name="SMA30"), row = 1, col = 1)
    fig.add_trace(go.Line(x = df.index, y = df['SMA10'], line=dict(color="orange", width=1), name="SM10"), row = 1, col = 1)
   
    #  Plot Bull/Bear market
    fig.add_trace(go.Line(x = df.index, y = df['BULL'], line=dict(color="green", width=1), name="Bullish Trend"), row = 1, col = 1)
    fig.add_trace(go.Line(x = df.index, y = df['BEAR'], line=dict(color="red", width=1), name="Bearish Trend"), row = 1, col = 1)
    
    #  Plot Coins
    #fig.add_trace(go.Line(x = df.index, y = df['Close_STX-USD'], line=dict(color="black", width=1), name="STX-USD"), row = 1, col = 1)
    #fig.add_trace(go.Line(x = df.index, y = df['Close_DOGE-USD'], line=dict(color="yellow", width=1), name="DOGE-USD"), row = 1, col = 1)
    #fig.add_trace(go.Line(x = df.index, y = df['Close_XLM-USD'], line=dict(color="purple", width=1), name="XLM-USD"), row = 1, col = 1)

    fig.update_layout(
        title={'text':'Primary Markets', 'x':0.5},
        autosize=False,
        width=800,height=400)
    fig.update_yaxes(range=[0,1000000000],secondary_y=True)
    fig.update_yaxes(visible=False, secondary_y=True)  #hide range slider
    
    fig.show()

In [111]:
plot_crypto_index(crypto_df)

C:\Users\dutch\anaconda3\envs\tradesystem1\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


